- pandas `DataFrame` and reading `CSV` from `SOS` URLs
- interactive plots (`hvplot`)
- interactive maps (`folium`)

In [1]:
import os

import geopandas

import pandas as pd

import wget


def load_best_track(code='al14', year='2018'):
    fname = f'{code}{year}_best_track.zip'
    url = f'https://www.nhc.noaa.gov/gis/best_track/{fname}'

    if not os.path.isfile(fname):
        import wget
        fname = wget.download(url)

    os.environ['CPL_ZIP_ENCODING'] = 'UTF-8'

    radii = geopandas.read_file(
        f'/{code.upper()}{year}_radii.shp',
        vfs='zip://{}'.format(fname)
    )

    pts = geopandas.read_file(
        f'/{code.upper()}{year}_pts.shp',
        vfs='zip://{}'.format(fname)
    )
    radii.index = pd.to_datetime(radii['SYNOPTIME'], format='%Y%m%d%H', errors='coerce').values
    return radii, pts

radii, pts = load_best_track(code='al14', year='2018')

bbox = radii['geometry'].total_bounds

In [2]:
from datetime import datetime


start = datetime(2018, 10, 7)
end = datetime(2018, 10, 16)

variable = 'water_surface_height_above_reference_datum'

buoy = '8728690'

url = (
    'https://opendap.co-ops.nos.noaa.gov/ioos-dif-sos/SOS?'
    'service=SOS'
    '&request=GetObservation'
    '&version=1.0.0'
    f'&observedProperty={variable}'
    f'&offering=urn:ioos:station:NOAA.NOS.CO-OPS:{buoy}'
    '&responseFormat=text/csv'
    f'&eventTime={start:%Y-%m-%dT%H:%M:%SZ}/'
    f'{end:%Y-%m-%dT%H:%M:%SZ}'
    '&result=VerticalDatum==urn:ogc:def:datum:epsg::5103'
    '&dataType=PreliminarySixMinute'
)

In [3]:
print(url)

https://opendap.co-ops.nos.noaa.gov/ioos-dif-sos/SOS?service=SOS&request=GetObservation&version=1.0.0&observedProperty=water_surface_height_above_reference_datum&offering=urn:ioos:station:NOAA.NOS.CO-OPS:8728690&responseFormat=text/csv&eventTime=2018-10-07T00:00:00Z/2018-10-16T00:00:00Z&result=VerticalDatum==urn:ogc:def:datum:epsg::5103&dataType=PreliminarySixMinute


In [4]:
import pandas as pd


df = pd.read_csv(
    url,
    index_col='date_time',
    parse_dates=True
)

df.head()

,station_id,sensor_id,latitude (degree),longitude (degree),water_surface_height_above_reference_datum (m),datum_id,vertical_position (m),sigma,quality_flags
date_time,,,,,,,,,
2018-10-07 00:00:00,urn:ioos:station:NOAA.NOS.CO-OPS:8728690,urn:ioos:sensor:NOAA.NOS.CO-OPS:8728690:A1,29.7244,-84.9806,0.153,urn:ogc:def:datum:epsg::5103,1.539,0.001,1;0;0;0
2018-10-07 00:06:00,urn:ioos:station:NOAA.NOS.CO-OPS:8728690,urn:ioos:sensor:NOAA.NOS.CO-OPS:8728690:A1,29.7244,-84.9806,0.147,urn:ogc:def:datum:epsg::5103,1.539,0.001,1;0;0;0
2018-10-07 00:12:00,urn:ioos:station:NOAA.NOS.CO-OPS:8728690,urn:ioos:sensor:NOAA.NOS.CO-OPS:8728690:A1,29.7244,-84.9806,0.141,urn:ogc:def:datum:epsg::5103,1.539,0.001,0;0;0;0
2018-10-07 00:18:00,urn:ioos:station:NOAA.NOS.CO-OPS:8728690,urn:ioos:sensor:NOAA.NOS.CO-OPS:8728690:A1,29.7244,-84.9806,0.132,urn:ogc:def:datum:epsg::5103,1.539,0.001,0;0;0;0
2018-10-07 00:24:00,urn:ioos:station:NOAA.NOS.CO-OPS:8728690,urn:ioos:sensor:NOAA.NOS.CO-OPS:8728690:A1,29.7244,-84.9806,0.126,urn:ogc:def:datum:epsg::5103,1.539,0.001,0;0;0;0


In [5]:
def extract_metadata(col):
    value = col.unique()
    if len(value) > 1:
        raise ValueError(
            f'Expected a single value but got {len(value)}'
        )
    return value.squeeze().tolist()

In [6]:
col = df.columns[df.columns.str.startswith(variable)]

idxmax = df[col].idxmax().squeeze()
dedup = radii.loc[~radii.index.duplicated(keep='first')]
overlap = dedup.iloc[dedup.index.get_loc(idxmax, method='nearest')]

In [7]:
import folium


sensor_id = extract_metadata(df['sensor_id'])

location = (
    extract_metadata(df['latitude (degree)']),
    extract_metadata(df['longitude (degree)'])
)


m = folium.Map(location=location, zoom_start=5)
folium.Marker(location=location, popup=sensor_id).add_to(m)

for geom in radii['geometry']:
    folium.GeoJson(geom.__geo_interface__).add_to(m)


style_function = lambda feature: {
    'fillColor': '#FF5733',
    'opacity': '0.15'
}

folium.GeoJson(overlap['geometry'].__geo_interface__, style_function=style_function).add_to(m)
m

In [8]:
import hvplot.pandas


df[col].hvplot.line(
    figsize=(9, 2.75),
    legend=False,
    grid=True,
    title=sensor_id,
)

:Curve   [date_time]   (water_surface_height_above_reference_datum (m))

IOOS maintains a Python library for collecting Met/Ocean observations named `pyoos` that aims the make it easier to access data from services like:

-  IOOS SWE SOS 1.0 Services
-  NERRS Observations - SOAP
-  NDBC Observations - SOS
-  CO-OPS Observations - SOS
-  STORET Water Quality - WqxOutbound via REST (waterqualitydata.us)
-  USGS NWIS Water Quality - WqxOutbound via REST (waterqualitydata.us)
-  USGS Instantaneous Values - WaterML via REST
-  NWS AWC Observations - XML via REST (http://www.aviationweather.gov)
-  HADS (http://www.nws.noaa.gov/oh/hads/ - limited to 7 day rolling
   window of data)

More information on https://github.com/ioos/pyoos